## Routes

The backend exposes several routes for managing instances and solutions.

- `GET /{PROBLEM_UID}/instances/{INSTANCE_UID}`: Retrieve a specific instance for a problem by its UID.
- `GET /{PROBLEM_UID}/instance_info`: Query instance metadata with pagination and filtering support.
- `GET /{PROBLEM_UID}/instance_schema`: Return the JSON schema of the instance model.
- `GET /{PROBLEM_UID}/instance_info/{INSTANCE_UID}`: Retrieve metadata for a specific instance.
- `GET /{PROBLEM_UID}/problem_info`: Retrieve metadata about the problem, including filters and asset information.
- `POST /{PROBLEM_UID}/instances/`: Create a new instance and index it for querying. This is protected by an ApiKey, which needs to be provided in the request header as `api_key`.
- `DELETE /{PROBLEM_UID}/instances/{INSTANCE_UID}`: Delete a specific instance by its UID.

### Assets

Instances can have associated assets (e.g., images or thumbnails). The backend provides endpoints to manage these (optional) assets.
Note that the assets will be served by the nginx server, and these endpoints are just for managing the assets or getting the asset paths.

- `POST /{PROBLEM_UID}/assets/{ASSET_CLASS}/{INSTANCE_UID}`: Upload an asset (e.g., image or thumbnail) for a specific instance. This is protected by an ApiKey, which needs to be provided in the request.
- `GET /{PROBLEM_UID}/assets/{INSTANCE_UID}`: Retrieve all assets associated with a specific instance. The response includes the asset classes and their corresponding file paths.
- `DELETE /{PROBLEM_UID}/assets/{ASSET_CLASS}/{INSTANCE_UID}`: Delete a specific asset for an instance. This is protected by an ApiKey, which needs to be provided in the request.

### Solutions (Optional)

If solutions are configured for a problem class, the backend provides endpoints to manage solutions.

- `GET /{PROBLEM_UID}/solutions/{SOLUTION_UID}`: Retrieve a specific solution by its UID.
- `GET /{PROBLEM_UID}/solution_info/{INSTANCE_UID}`: Retrieve paginated solution metadata for a specific instance.
- `GET /{PROBLEM_UID}/solution_schema`: Retrieve the JSON schema of the solution model.
- `POST /{PROBLEM_UID}/solutions`: Enter a new solution for a specific instance. The solution must reference a valid instance UID. This is protected by an ApiKey, which needs to be provided in the request header as `api_key`.
- `DELETE /{PROBLEM_UID}/solutions/{SOLUTION_UID}`: Delete a specific solution by its UID. This operation also removes the solution from the index. This is protected by an ApiKey, which needs to be provided in the request header as `api_key`.

In [100]:
from pydantic import (
    BaseModel,
    Field,
    NonNegativeFloat,
    NonNegativeInt,
    PositiveFloat,
    PositiveInt,
)


class KnapsackInstance(BaseModel):
    """Pydantic model representing a knapsack problem instance."""

    # Metadata
    instance_uid: str = Field(..., description="The unique identifier of the instance")
    origin: str = Field(default="", description="The origin or source of the instance")

    # Instance statistics
    num_items: PositiveInt = Field(
        ..., description="The number of items available in the instance"
    )
    weight_capacity_ratio: PositiveFloat = Field(
        ...,
        description=(
            "The ratio of the total weight of all items to the knapsack capacity. "
            "Calculated as the sum of item weights divided by knapsack capacity."
        ),
    )
    integral: bool = Field(
        default=False,
        description="Specifies if the capacity, values, and weights are integral.",
    )

    # Instance data
    capacity: NonNegativeFloat | NonNegativeInt = Field(
        ..., description="The total capacity of the knapsack"
    )
    item_values: list[NonNegativeFloat | NonNegativeInt] = Field(
        ..., description="The values assigned to each item"
    )
    item_weights: list[NonNegativeFloat | NonNegativeInt] = Field(
        ..., description="The weights assigned to each item"
    )


class KnapsackSolution(BaseModel):
    """Pydantic model representing a solution to a knapsack problem instance."""

    # Solution metadata
    instance_uid: str = Field(
        ..., description="The unique identifier of the corresponding instance"
    )
    objective: NonNegativeFloat = Field(
        ..., description="The objective value of the solution (e.g., total value)"
    )
    authors: str = Field(..., description="The authors or contributors of the solution")

    # Solution data
    selected_items: list[int] = Field(
        ..., description="Indices of the selected items in the solution"
    )


# Configuration constants for the knapsack problem

# Unique identifier for the problem
PROBLEM_UID = "knapsack"

# Shared attribute name for instances and solutions
INSTANCE_UID_ATTRIBUTE = "instance_uid"

# Schema definitions
INSTANCE_SCHEMA = KnapsackInstance
SOLUTION_SCHEMA = (
    KnapsackSolution  # Optional: Set to None if solutions are not supported
)

# Filtering and sorting configurations
RANGE_FILTERS = [
    "num_items",
    "weight_capacity_ratio",
]  # Fields usable for range filters
BOOLEAN_FILTERS = ["integral"]  # Boolean fields usable for filters
SORT_FIELDS = ["num_items", "weight_capacity_ratio"]  # Fields usable for sorting

# Fields for display purposes in instance overviews
DISPLAY_FIELDS = [
    "instance_uid",
    "num_items",
    "weight_capacity_ratio",
    "integral",
    "origin",
]

# Assets associated with the knapsack problem
ASSETS = {"thumbnail": "png", "image": "png"}

# Solution-specific configurations
SOLUTION_SORT_ATTRIBUTES = [
    "objective"
]  # Fields for sorting solutions. A "-" prefix indicates descending order.
SOLUTION_DISPLAY_FIELDS = ["objective", "authors"]  # Fields to display for solutions


In [141]:
import requests

class Connector:
    def __init__(self, base_url: str, problem_uid: str, api_key: str|None = None):
        self.base_url = base_url
        self.problem_uid = problem_uid
        self.api_key = api_key

    def get_instance_schema(self):
        """Returns the schema for problem instances."""
        response = requests.get(
            f"{self.base_url}/{self.problem_uid}/instance_schema")
        response.raise_for_status()
        return response.json()
    
    def get_instance(self, instance_uid: str):
        """Fetches a specific problem instance by its UID."""
        response = requests.get(
            f"{self.base_url}/{self.problem_uid}/instances/{instance_uid}")
        response.raise_for_status()
        return response.json()
    
    def get_all_instance_info(self, offset: int = 0, limit: int = 100, params: dict|None = None):
        """Fetches all problem instances."""
        if params is None:
            params = {}
        response = requests.get(f"{self.base_url}/{self.problem_uid}/instance_info", 
                                params={"offset": offset, "limit": limit}.update(params))
        response.raise_for_status()
        return response.json()
    
    def get_instance_info(self, instance_uid: str):
        """Fetches information about a specific problem instance."""
        response = requests.get(
            f"{self.base_url}/{self.problem_uid}/instance_info/{instance_uid}")
        response.raise_for_status()
        return response.json()
    
    def get_problem_info(self):
        """Fetches information about the problem."""
        response = requests.get(f"{self.base_url}/{self.problem_uid}/problem_info/")
        response.raise_for_status()
        return response.json()
    
    def upload_instance(self, instance: BaseModel):
        """Uploads a new problem instance."""
        headers = {}
        if self.api_key:
            headers["api-key"] = self.api_key
            
        response = requests.post(
            f"{self.base_url}/{self.problem_uid}/instances",
            json=instance.model_dump(mode="json"),
            headers=headers
        )
        response.raise_for_status()
        return response.json()
    
    def delete_instance(self, instance_uid: str):
        """Deletes a problem instance by its UID."""
        headers = {}
        if self.api_key:
            headers["api-key"] = self.api_key
            
        response = requests.delete(
            f"{self.base_url}/{self.problem_uid}/instances/{instance_uid}",
            headers=headers
        )
        response.raise_for_status()
        return response.json()
    
    # assets
    def get_assets(self, instance_uid: str):
        """Fetches all assets for a problem instance."""
        response = requests.get(
            f"{self.base_url}/{self.problem_uid}/assets/{instance_uid}")
        response.raise_for_status()
        return response.json()
    
    def upload_asset(self, instance_uid: str, asset_class: str, asset_path: str):
        """Uploads an asset for a problem instance."""
        headers = {}
        if self.api_key:
            headers["api-key"] = self.api_key
        
        # Read the file data from the provided path
        with open(asset_path, 'rb') as f:
            files = {'file': f}
            
            response = requests.post(
            f"{self.base_url}/{self.problem_uid}/assets/{asset_class}/{instance_uid}",
            files=files,
            headers=headers
            )
            print(response.text)
            
        response.raise_for_status()
        return response.json()
    
    def delete_asset(self, instance_uid: str, asset_class: str):
        """Deletes a specific asset for a problem instance."""
        headers = {}
        if self.api_key:
            headers["api-key"] = self.api_key
            
        response = requests.delete(
            f"{self.base_url}/{self.problem_uid}/assets/{asset_class}/{instance_uid}",
            headers=headers
        )
        response.raise_for_status()
        return response.json()
    
    # solutions
    def get_solution_schema(self):
        """Returns the schema for problem solutions."""
        response = requests.get(
            f"{self.base_url}/{self.problem_uid}/solution_schema")
        response.raise_for_status()
        return response.json()
    
    def get_solution_info(self, instance_uid: str, offset: int = 0, limit: int = 100):
        """Fetches the solutions for a specific problem instance."""
        response = requests.get(
            f"{self.base_url}/{self.problem_uid}/solution_info/{instance_uid}", 
            params={"offset": offset, "limit": limit})
        response.raise_for_status()
        return response.json()
    
    def upload_solution(self, solution: BaseModel):
        """Uploads a new solution for a problem instance."""
        headers = {}
        if self.api_key:
            headers["api-key"] = self.api_key
            
        response = requests.post(
            f"{self.base_url}/{self.problem_uid}/solutions",
            json=solution.model_dump(mode="json"),
            headers=headers
        )
        response.raise_for_status()
        return response.json()
    
    def get_solution(self, solution_uid: str):
        """Fetches a specific solution by its UID."""
        response = requests.get(
            f"{self.base_url}/{self.problem_uid}/solutions/{solution_uid}")
        response.raise_for_status()
        return response.json()
    
    def delete_solution(self, solution_uid: str):
        """Deletes a specific solution for a problem instance."""
        headers = {}
        if self.api_key:
            headers["api-key"] = self.api_key
            
        response = requests.delete(
            f"{self.base_url}/{self.problem_uid}/solutions/{solution_uid}",
            headers=headers
        )
        response.raise_for_status()
        return response.json()
    
# For the local example configuration    
connector = Connector(base_url="http://127.0.0.1", problem_uid=PROBLEM_UID, api_key="3456345-456-456")


## Get Problem Metadata

In [ ]:
connector.get_problem_info()

{'problem_uid': 'knapsack',
 'range_filters': [{'field_name': 'num_items',
   'problem_uid': 'knapsack',
   'min_val': 13.0,
   'max_val': 1000.0},
  {'field_name': 'weight_capacity_ratio',
   'problem_uid': 'knapsack',
   'min_val': 0.001174003923964717,
   'max_val': 0.9805904732310761}],
 'boolean_filters': ['integral'],
 'sort_fields': ['num_items', 'weight_capacity_ratio'],
 'display_fields': ['instance_uid',
  'num_items',
  'weight_capacity_ratio',
  'integral',
  'origin'],
 'assets': {'thumbnail': 'png', 'image': 'png'}}

## Generate and upload some instances

In [111]:
# generate random instances for testing
from random import randint, random
from uuid import uuid4

instances = []
for _ in range(10):
    num_items = randint(10, 1000)
    weight_capacity_ratio = random()
    integral = random() < 0.5
    capacity = random() * 100
    item_values = [random() * 100 for _ in range(num_items)]
    item_weights = [random() * 100 for _ in range(num_items)]
    if integral:
        item_values = [round(v) for v in item_values]
        item_weights = [round(w) for w in item_weights]
        capacity = round(capacity)
    instance = KnapsackInstance(
        instance_uid="test/"+str(uuid4()),
        num_items=num_items,
        weight_capacity_ratio=weight_capacity_ratio,
        integral=integral,
        capacity=capacity,
        item_values=item_values,
        item_weights=item_weights,
    )
    connector.upload_instance(instance)
    


In [124]:
all_instances = connector.get_all_instance_info()
all_instances

{'sorted_uids': ['test/04168d94-9307-4824-bcdf-b45e62d5ea79',
  'test/f6087360-cf50-4a0b-b296-4a65c657fbc4',
  'test/07692979-6b57-4e19-b220-d1b03e864d09',
  'test/b9214f20-5f5f-45e5-95b4-c46f43e8256b',
  'test/fabb4897-7a52-4887-9f2e-a6b81bb4d94a',
  'test/d01c8724-e3ee-4ed8-bc5b-c5cc7cf57448',
  'test/0f5bf1cc-0fba-47fa-9c6a-035c07733edf',
  'test/8bb2db9f-79f1-4540-a989-136b74917c3c',
  'test/ad11c055-67df-4c7a-a646-2a7a22fb08a4',
  'test/2173058b-37d7-44b3-afc3-31f7611456c2'],
 'data': {'test/04168d94-9307-4824-bcdf-b45e62d5ea79': {'weight_capacity_ratio': 0.3624288186473066,
   'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
   'origin': '',
   'integral': 0,
   'num_items': 264},
  'test/f6087360-cf50-4a0b-b296-4a65c657fbc4': {'weight_capacity_ratio': 0.9828961048960384,
   'instance_uid': 'test/f6087360-cf50-4a0b-b296-4a65c657fbc4',
   'origin': '',
   'integral': 0,
   'num_items': 144},
  'test/07692979-6b57-4e19-b220-d1b03e864d09': {'weight_capacity_ratio': 0.131

In [125]:
random_instance_uid = all_instances['sorted_uids'][0]

In [133]:
connector.get_instance_info(random_instance_uid)

{'weight_capacity_ratio': 0.3624288186473066,
 'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
 'origin': '',
 'integral': 0,
 'num_items': 264}

In [126]:
connector.get_instance(random_instance_uid)

{'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
 'origin': '',
 'num_items': 264,
 'weight_capacity_ratio': 0.3624288186473066,
 'integral': False,
 'capacity': 50.77823662739207,
 'item_values': [11.47169459324372,
  16.24368535341195,
  11.135907388842492,
  38.904318593348165,
  96.65672658755959,
  98.74899623847747,
  92.4780096275241,
  96.395516514061,
  90.17070856343516,
  45.02778602763004,
  49.982329226643266,
  36.06335165215564,
  88.00319425607611,
  70.17680112271637,
  29.44651940767319,
  34.98589778265935,
  87.85296029989179,
  30.498206303492246,
  50.7834114817285,
  14.503041505467317,
  98.16041637797179,
  83.86567668593713,
  81.76197361125591,
  52.365610370073036,
  83.99034756394754,
  34.27381790534986,
  45.57456912983091,
  25.518052731829943,
  55.53294941773645,
  71.83370187179902,
  43.318503051611536,
  49.71449082267026,
  56.582803899791635,
  39.61433967799343,
  42.389695426542275,
  65.93055636487655,
  29.856792417043465,
  28.22

## Assets

In [128]:
# create random png via matplotlib
import matplotlib.pyplot as plt
def create_random_png():
    """Generates a random PNG image."""
    data = [[random() for _ in range(10)] for _ in range(10)]
    plt.imshow(data, cmap='viridis', interpolation='nearest')
    plt.axis('off')
    plt.savefig('random_image.png', format='png', bbox_inches='tight', pad_inches=0)
    plt.close()

create_random_png()

connector.upload_asset(
    instance_uid=random_instance_uid,
    asset_class="thumbnail",
    asset_path="random_image.png"
)

null


In [129]:
connector.get_assets(random_instance_uid)

{'thumbnail': 'http://127.0.0.1:80/static/knapsack/assets/thumbnail/test/04168d94-9307-4824-bcdf-b45e62d5ea79.png'}

In [131]:
connector.delete_asset(
    instance_uid=random_instance_uid,
    asset_class="thumbnail"
)

In [132]:
connector.get_assets(random_instance_uid)

{}

## Solutions

In [138]:
connector.get_solution_schema()

{'description': 'Pydantic model representing a solution to a knapsack problem instance.',
 'properties': {'instance_uid': {'description': 'The unique identifier of the corresponding instance',
   'title': 'Instance Uid',
   'type': 'string'},
  'objective': {'description': 'The objective value of the solution (e.g., total value)',
   'minimum': 0,
   'title': 'Objective',
   'type': 'number'},
  'authors': {'description': 'The authors or contributors of the solution',
   'title': 'Authors',
   'type': 'string'},
  'selected_items': {'description': 'Indices of the selected items in the solution',
   'items': {'type': 'integer'},
   'title': 'Selected Items',
   'type': 'array'}},
 'required': ['instance_uid', 'objective', 'authors', 'selected_items'],
 'title': 'KnapsackSolution',
 'type': 'object'}

In [139]:
connector.get_solution_info(
    instance_uid=random_instance_uid
)

{'items': [], 'offset': 0, 'limit': 100, 'total': 0}

In [143]:
connector.upload_solution(
    KnapsackSolution(
        instance_uid=random_instance_uid,
        objective=100.0,
        authors="John Doe",
        selected_items=[0, 1, 2]
    )
)

In [144]:
solution_info = connector.get_solution_info(
    instance_uid=random_instance_uid
)
solution_info

{'items': [{'objective': 100.0,
   'solution_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79/a8d100c21cd5b653cd831f2c846817b0',
   'authors': 'John Doe',
   'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79'}],
 'offset': 0,
 'limit': 100,
 'total': 1}

In [146]:
random_solution_uid = solution_info['items'][0]['solution_uid']

In [147]:
connector.get_solution(solution_uid=random_solution_uid)

{'instance_uid': 'test/04168d94-9307-4824-bcdf-b45e62d5ea79',
 'objective': 100.0,
 'authors': 'John Doe',
 'selected_items': [0, 1, 2]}

In [148]:
connector.delete_solution(solution_uid=random_solution_uid)